In [1]:
%matplotlib widget

import numpy as np

In [2]:
from colour import Color
import matplotlib.pyplot as plt


class ColorGradient:
    def __init__(self, resolution=100):
        low_color = Color("red")
        high_color = Color("blue")
        
        self.size = resolution
        self.colors = list(low_color.range_to(high_color, self.size))
    
    def get(self, value):
        """value: 0.0 ~ 1.0"""
        i = int(value * self.size)
        if i >= self.size:
            i = self.size - 1
        r, g, b = self.colors[i].rgb
        return (r, g, b, 1.0)

def scale_remapping(value, max_value, min_value, max_target, min_target):
    gain = (value - min_value) / (max_value - min_value)
    return min_target + gain * (max_target - min_target)

def filtering(raw):
    return raw[:,[IDX_Y, IDX_TCP_X, IDX_TCP_Y, IDX_M]]

def scatter_2d(ax, raw, xyMinMax, is_irm=True, dot_size=1, title=None):
    """
    raw:
        [[Cr x y manip],
         [Cr x y manip], ...]
         deg m m -
     
    Example:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
        xyminmax = [-0.8, 0.8, -0.8, 0.8]
        scatter_2d(ax1, filtering(rm), xyminmax, is_irm=False)
        scatter_2d(ax2, filtering(irm), xyminmax)
        fig.tight_layout()
        plt.show()
    """
    Crs, xs, ys, ms = np.transpose(raw)
    max_manip = np.max(ms)
    min_manip = np.min(ms)
    grad = ColorGradient()
    colors = np.array(
        [grad.get(scale_remapping(m, max_manip, min_manip, 1, 0)) for m in ms]
    )
    
    ax.scatter(xs, ys, c=colors, s=dot_size)
    
    title = title if title else ("IRM" if is_irm else "RM")
    min_Cr = np.max(Crs)
    max_Cr = np.min(Crs)
    ax.set_title("%s ($C_{r}$ range: %.1f~%.1f deg)" % (title, min_Cr, max_Cr))
    ax.set_xlabel("x(m)", fontsize=15)
    ax.set_ylabel("y(m)", fontsize=15)
    ax.axis(xyMinMax)
    ax.grid(True)

    # robot and object
    if is_irm:  # object
        marker_x = [1.0, -0.5, -0.5, -0.5]
        marker_y = [0.0, 0.0, -0.5, 0.5]
    else:  # robot
        marker_x = [1.0, -0.5, -0.5, 1.0]
        marker_y = [0.0, 0.5, -0.5, 0.0]
    arrow_size = max(xyMinMax) * 0.1
    arrow_x = np.array(marker_x) * arrow_size
    arrow_y = np.array(marker_y) * arrow_size
    ax.plot(arrow_x, arrow_y)

def scatter_3d(ax, raw, xyMinMax, is_irm=True, dot_size=2, title=None):
    """
    raw:
        [[Cr x y manip],
         [Cr x y manip], ...]
         deg m m -
     
    Example:
        fig = plt.figure()
        ax1 = fig.add_subplot(121, projection='3d')
        ax2 = fig.add_subplot(122, projection='3d')
        xyminmax = [-0.8, 0.8, -0.8, 0.8]
        scatter_3d(ax1, filtering(rm), xyminmax, is_irm=False)
        scatter_3d(ax2, filtering(irm), xyminmax)
        plt.show()
    """
    Crs, xs, ys, ms = np.transpose(raw)
    max_manip = np.max(ms)
    min_manip = np.min(ms)
    grad = ColorGradient()
    colors = np.array(
        [grad.get(scale_remapping(m, max_manip, min_manip, 1, 0)) for m in ms]
    )
    
    ax.scatter(xs, ys, zs=Crs, c=colors, s=dot_size)
    ax.set_xlabel("x(m)", fontsize=15)
    ax.set_ylabel("y(m)", fontsize=15)
    ax.set_zlabel("$C_{r}$(deg)", fontsize=15)
    ax.axis(xyMinMax)
    
    title = title if title else ("IRM" if is_irm else "RM")
    min_Cr = np.max(Crs)
    max_Cr = np.min(Crs)
    ax.set_title("%s ($C_{r}$ range: %.1f~%.1f deg)" % (title, min_Cr, max_Cr))

    # robot and object
    if is_irm:  # object
        marker_x = [1.0, -0.5, -0.5, -0.5]
        marker_y = [0.0, 0.0, -0.5, 0.5]
    else:  # robot
        marker_x = [1.0, -0.5, -0.5, 1.0]
        marker_y = [0.0, 0.5, -0.5, 0.0]
    arrow_size = max(xyMinMax) * 0.1
    arrow_x = np.array(marker_x) * arrow_size
    arrow_y = np.array(marker_y) * arrow_size
    
    int_Crs = set(np.array(Crs, dtype=int))
    for c in int_Crs:
        arrow_z = np.array([c, c, c, c])
        ax.plot(arrow_x, arrow_y, arrow_z)

CSV to NPY

```py
import csv

raw_file = "robocare_right_reachability_map.csv"
first_row_is_title = True
is_rpy_order = False

raw = []
with open(raw_file, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    raw = [row for row in reader]
    if first_row_is_title:
        raw = raw[1:]

npraw = np.array(raw, dtype=np.float)
# Cr,pitch,roll -> roll,pitch,Cr
npraw[:,[6, 7, 8]] = npraw[:,[8, 7, 6]]

print(npraw.shape)
```

# RM to IRM

Transform the reachability map(RM) to the inverse RM. Each data point will be rotated by Cr about the Z-axis.

**Table of Contents**
1. Load data
2. RM to IRM
3. Check with plots
4. Save IRM

### 1. Load data

| Column Index | Name              | Unit       | Remark |
| ----- | ----------------- | ---------- | ------ |
| 0     | TCP x             | meter      |        |
| 1     | TCP y             | meter      | L!=R   |
| 2     | TCP z             | meter      |        |
| 3     | EEP x             | meter      |        |
| 4     | EEP y             | meter      | L!=R   |
| 5     | EEP z             | meter      |        |
| 6     | EE Roll           | **DEGREE** |        |
| 7     | EE Pitch          | **DEGREE** |        |
| 8     | EE Yaw: Cr        | **DEGREE** | L!=R   |
| 9     | Manipulability    | -          |        |
| 10    | Joint_0: Waist_Roll      | radian     | L!=R   |
| 11    | Joint_1: Waist_Pitch     | radian     |        |
| 12    | Joint_2: RShoulder_Pitch | radian     |        |
| 13    | Joint_3: RShoulder_Roll  | radian     | L!=R   |
| 14    | Joint_4: RElbow_Pitch    | radian     |        |
| 15    | Joint_5: RElbow_Yaw      | radian     | L!=R   |
| 16    | Joint_6: RWrist_Pitch    | radian     |        |
| 17    | Joint_7: RWrist_Roll     | radian     | L!=R   |

"robocare_right_reachability_map.npy" has 18 columns. (Index 17 is "Joint_7".)

In [3]:
IDX_TCP_X = 0
IDX_TCP_Y = 1
IDX_TCP_Z = 2
IDX_EEP_X = 3
IDX_EEP_Y = 4
IDX_EEP_Z = 5
IDX_R = 6
IDX_P = 7
IDX_Y = 8
IDX_M = 9

# right_rm = np.load('robocare_right_reachability_map.npy')
right_rm = np.load('socialrobot_RM_2021-10-14_09-50-24_wip.npy')
print(right_rm.shape)
print(right_rm[:2])

(25713, 18)
[[ 4.00000000e-01 -2.00000000e-01  7.50000000e-01  3.33251405e-01
  -2.38537320e-01  6.72925361e-01 -9.00000000e+01  4.50000000e+01
   3.00000000e+01  5.30626228e-02  2.23071169e-01 -4.72555941e-01
   2.13756462e+00 -8.07449947e-01 -2.19218348e+00  1.35919661e+00
  -2.29378927e+00  9.99775621e-01]
 [ 4.00000000e-01 -2.00000000e-01  7.50000000e-01  3.22925361e-01
  -2.00000000e-01  6.72925361e-01 -9.00000000e+01  4.50000000e+01
   0.00000000e+00  6.68470906e-02  2.96737074e-01 -4.39959976e-01
  -6.60437348e-01  1.48795512e+00  3.59207843e-02  1.42871093e+00
  -1.36285730e+00  7.05210566e-01]]


### Right to Left

## 2. RM to IRM

Linear Transformation (only rotation)

In [5]:
# rm = left_rm
rm = right_rm
rm.shape

(25713, 18)

In [6]:
import copy

def rotation_2d_about_z(rad):
    c, s = np.cos(rad), np.sin(rad)
    R = np.array((
        (c, -s),
        (s, c)
    ))
    return R

def rm_to_irm(rm):
    irm = copy.deepcopy(rm)
    for original_row in irm:
        row = copy.deepcopy(original_row)
        Cr = row[IDX_Y]  # Degree
        R = rotation_2d_about_z(np.radians(180. - Cr))
        
        original_row[IDX_TCP_X:IDX_TCP_Z] = np.dot(R, row[IDX_TCP_X:IDX_TCP_Z])
    return irm
        
irm = rm_to_irm(rm)
print(irm.shape)
print(irm[:2])

(25713, 18)
[[-2.46410162e-01  3.73205081e-01  7.50000000e-01  3.33251405e-01
  -2.38537320e-01  6.72925361e-01 -9.00000000e+01  4.50000000e+01
   3.00000000e+01  5.30626228e-02  2.23071169e-01 -4.72555941e-01
   2.13756462e+00 -8.07449947e-01 -2.19218348e+00  1.35919661e+00
  -2.29378927e+00  9.99775621e-01]
 [-4.00000000e-01  2.00000000e-01  7.50000000e-01  3.22925361e-01
  -2.00000000e-01  6.72925361e-01 -9.00000000e+01  4.50000000e+01
   0.00000000e+00  6.68470906e-02  2.96737074e-01 -4.39959976e-01
  -6.60437348e-01  1.48795512e+00  3.59207843e-02  1.42871093e+00
  -1.36285730e+00  7.05210566e-01]]


## 3. Check

### 2D & 3D Plot

In [7]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
xyminmax = [-0.8, 0.8, -0.8, 0.8]
scatter_2d(ax1, filtering(rm), xyminmax, is_irm=False)
scatter_2d(ax2, filtering(irm), xyminmax)
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig = plt.figure()
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
xyminmax = [-0.8, 0.8, -0.8, 0.8]
scatter_3d(ax1, filtering(rm), xyminmax, is_irm=False)
scatter_3d(ax2, filtering(irm), xyminmax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 4. Save IRM

In [10]:
npy_name = "socialrobot_wip_right_irm.npy"
np.save(npy_name, irm)


### IRM Data (Input)

- Sample: ./config/robocare_right_irm.npy
  - This sample has 7899 points.
  - The supported z of target is {0.7, 0.75, 0.8}.

| Column Index | Name                     | Unit       | Remark           |
| ------------ | ------------------------ | ---------- | ---------------- |
| 0            | Mobile Base x            | meter      | For query output |
| 1            | Mobile Base y            | meter      | For query output |
| 2            | Target Object z (height) | meter      | For query input  |
| 3            | EEP x                    | meter      | For IK solver    |
| 4            | EEP y                    | meter      | For IK solver    |
| 5            | EEP z                    | meter      | For IK solver    |
| 6            | EE Roll                  | **DEGREE** |                  |
| 7            | EE Pitch                 | **DEGREE** |                  |
| 8            | EE Yaw                   | **DEGREE** | Cr               |
| 9            | Manipulability           | -          |                  |
| 10           | Joint_0 value            | radian     |                  |
| 11           | Joint_1 value            | radian     |                  |
| >=12         | Joint_2... values        | radian     |                  |
